<a href="https://colab.research.google.com/github/ArthurAllioux/DataCleaning/blob/main/TP_DATACleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd 
import numpy as np
import seaborn as sns
import math
import json
import re
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

file_url='https://raw.githubusercontent.com/ManarOmar/New-York-Airbnb-2019/master/AB_NYC_2019.csv'
file_ori=pd.read_csv(file_url)
file_copy = file_ori.copy()
file_copy

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [28]:
age = [24, 24.7, 26, '26', '26.5', '25c', 'epf', 250, 0, -22]
zipcode = ['34000', 34070, 'epf', '34000c', 6000, '06000', '6000', '6000.0',
           99999, 3400]
date = ['1/1/21', '02-02-2021', '02-02/2021', '', 'epf', '1/2021', '2021',
        '1-1-1930', '1-1-2221', '31/02/2021']
price = [-3.4, 3.4, '3.4', '5 eur', '3000', '4 eur / item', 'epf', '-17.4',
         '-17.2', '-2.8 eur / item']
gps = [[44.7, 3.9845],[44.7, 3.9845], [44.7, 3.9845], [44.7, 3.9845],
       [44.7, 3.9845], [44.7, 3.9845],[44.7, 3.9845], 1, '[44.7, 3.9845]',
       [100, 200]]

In [29]:
ageSeries = pd.Series(age)
zipcodeSeries = pd.Series(zipcode)
dateSeries = pd.Series(date)
priceSeries = pd.Series(price)
gpsSeries = pd.Series(gps)

FakeDF = pd.DataFrame({'age':ageSeries,
                   'zipcode':zipcodeSeries,
                   'date':dateSeries,
                   'price': priceSeries,
                   'gps': gpsSeries
                  })
FakeDF


,age,zipcode,date,price,gps
0,24,34000,1/1/21,-3.4,"[44.7, 3.9845]"
1,24.7,34070,02-02-2021,3.4,"[44.7, 3.9845]"
2,26,epf,02-02/2021,3.4,"[44.7, 3.9845]"
3,26,34000c,,5 eur,"[44.7, 3.9845]"
4,26.5,6000,epf,3000,"[44.7, 3.9845]"
5,25c,06000,1/2021,4 eur / item,"[44.7, 3.9845]"
6,epf,6000,2021,epf,"[44.7, 3.9845]"
7,250,6000.0,1-1-1930,-17.4,1
8,0,99999,1-1-2221,-17.2,"[44.7, 3.9845]"
9,-22,3400,31/02/2021,-2.8 eur / item,"[100, 200]"


In [30]:
def remove_non_float(value):
    value = str(value)
    float_part_list = [ x for x in value if x.isdigit() or x in ['-','.']]
    float_part = ('').join(float_part_list)
    return float_part

def make_float(value):
    if value == '':
        str_as_float = np.nan
    else:
        str_as_float = float(value)  
    return str_as_float

def make_int(value):
    if not np.isnan(value):
        float_as_int = math.ceil(value)
    else:
        float_as_int=value
    return float_as_int

def format_age(dataframe, age_column_label='age'):
    dataframe[age_column_label]=dataframe[age_column_label].apply(remove_non_float)
    dataframe[age_column_label]=dataframe[age_column_label].apply(make_float)
    dataframe[age_column_label]=dataframe[age_column_label].apply(make_int)
    dataframe[age_column_label]=dataframe[age_column_label].astype('Int64')
    return dataframe


In [31]:
make_float('')

nan

In [32]:

def make_string(value):
    if value == '':
        int_as_str = np.nan
    else:
        int_as_str = str(value)  
    return int_as_str
    
def fill_5_characters_with_leading_zero(value):
    if isinstance(value, float):
        if np.isnan(value):
            zipcode = value
        else:
            zipcode = str(int(value)).zfill(5) 
    else:
        zipcode = str(int(value)).zfill(5)
    return zipcode

def format_zipcode(dataframe, zip_column_label='zipcode'):
    dataframe[zip_column_label]=dataframe[zip_column_label].apply(remove_non_float)
    dataframe[zip_column_label]=dataframe[zip_column_label].apply(make_float)
    dataframe[zip_column_label]=dataframe[zip_column_label].apply(make_int)
    #dataframe[zip_column_label]=dataframe[zip_column_label].astype('Int64')
    dataframe[zip_column_label] = dataframe[zip_column_label].apply(fill_5_characters_with_leading_zero)
    return dataframe

In [33]:
def format_date(dataframe, date_column_label='date'):
    dataframe[date_column_label] = pd.to_datetime(dataframe[date_column_label],errors='coerce')
    return dataframe

In [34]:
def format_price(dataframe, price_column_label='price'):
    dataframe[price_column_label]=dataframe[price_column_label].apply(remove_non_float)
    dataframe[price_column_label]=dataframe[price_column_label].apply(make_float)
    dataframe[price_column_label]=dataframe[price_column_label].astype('float64')
    return dataframe


In [35]:
def splitString(stringToSplit, separator):
      arrayOfStrings = stringToSplit.split(separator)
      return arrayOfStrings

def make_list(value):
    if isinstance(value, list):
        gps_list = value
    else:
        try:
            gps_list = json.loads(value)
            print(gps_list)
        except TypeError:
            gps_list = np.nan
    return gps_list
    
def get_latitude(gps_value):
    if isinstance(gps_value, list):
        return gps_value[0]
    else:
        return np.nan
        

def get_longitude(gps_value):
    if isinstance(gps_value, list):
        return gps_value[1]
    else:
        return np.nan

def format_gps(dataframe, gps_column_label='gps'):
    dataframe[gps_column_label] = dataframe[gps_column_label].apply(make_list)
    dataframe['latitude'] = dataframe[gps_column_label].apply(get_latitude)
    dataframe['longitude'] = dataframe[gps_column_label].apply(get_longitude)
    return dataframe

In [36]:
def format_DataFrame(dataframe):
    DataFrame = format_age(dataframe)
    DataFrame = format_zipcode(dataframe)
    DataFrame = format_date(dataframe)
    DataFrame = format_price(dataframe)
    DataFrame = format_gps(dataframe)


In [37]:

format_DataFrame(FakeDF)
FakeDF.zipcode


[44.7, 3.9845]


0    34000
1    34070
2      NaN
3    34000
4    06000
5    06000
6    06000
7    06000
8    99999
9    03400
Name: zipcode, dtype: object

In [38]:
target_df = pd.DataFrame({
    'age': [24, 25, 26, 26, 27, 25, np.nan, 250, 0, -22],
    'zipcode': ['34000', '34070', np.nan, '34000', '06000', '06000', '06000', '06000', '99999', '03400'],
    'date': [pd.to_datetime('2021-01-01'), pd.to_datetime('2021-02-02'), pd.to_datetime('2021-02-02'),
             np.nan, np.nan, pd.to_datetime('2021-01-01'), pd.to_datetime('2021-01-01'),
             pd.to_datetime('1930-01-01'), pd.to_datetime('2221-01-01'), np.nan,],
    'price': [-3.4, 3.4, 3.4, 5, 3000, 4, np.nan, -17.4, -17.2, -2.8],
    'gps': [[44.7, 3.9845],[44.7, 3.9845], [44.7, 3.9845], [44.7, 3.9845], [44.7, 3.9845],
            [44.7, 3.9845],[44.7, 3.9845], np.nan, [44.7, 3.9845], [100, 200]],
    'latitude': [44.7, 44.7, 44.7, 44.7, 44.7, 44.7, 44.7, np.nan, 44.7, 100],
    'longitude': [3.9845, 3.9845, 3.9845, 3.9845, 3.9845, 3.9845, 3.9845, np.nan, 3.9845, 200],
})
target_df

,age,zipcode,date,price,gps,latitude,longitude
0,24.0,34000,2021-01-01,-3.4,"[44.7, 3.9845]",44.7,3.9845
1,25.0,34070,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
2,26.0,NaN,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
3,26.0,34000,NaT,5.0,"[44.7, 3.9845]",44.7,3.9845
4,27.0,06000,NaT,3000.0,"[44.7, 3.9845]",44.7,3.9845
5,25.0,06000,2021-01-01,4.0,"[44.7, 3.9845]",44.7,3.9845
6,NaN,06000,2021-01-01,NaN,"[44.7, 3.9845]",44.7,3.9845
7,250.0,06000,1930-01-01,-17.4,NaN,NaN,NaN
8,0.0,99999,2221-01-01,-17.2,"[44.7, 3.9845]",44.7,3.9845
9,-22.0,03400,NaT,-2.8,"[100, 200]",100.0,200.0000


In [39]:
#target_df.equals(FakeDF)
target_df.gps.equals(FakeDF.gps)

True